In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io as sio
from scipy.stats.stats import pearsonr   
import scipy.stats as ss
import scipy.signal as ssg
from scipy.io import savemat
import time
import h5py
import mne
import timeit
import random
import glob  
import re

def spatial_cov(data):
    #input[ch time nstimuli]
    return(np.cov(data.reshape([data.shape[0],-1])))

def spatial_corr(data):
    #input[ch time nstimuli]
    return(np.corrcoef(data.reshape([data.shape[0],-1])))


def temporal_cov(data):
    #input[ch time nstimuli]
    data=np.swapaxes(data,1,0)
    return(np.cov(data.reshape([data.shape[0],-1])))

def temporal_corr(data):
    #input[ch time nstimuli]
    data=np.swapaxes(data,1,0)
    return(np.corrcoef(data.reshape([data.shape[0],-1])))


def compress_dim_by_n_nosti(data,dim,n):
    #assume data is of [nch ntime ]
    
    data_output=np.zeros([int(data.shape[0]/n),data.shape[1]])
    for i in range(data_output.shape[0]):
        data_output[i]=np.mean(data[i*n:(i+1)*n],axis=0)
   
    return(data_output)


def compress_dim_by_n(data,dim,n):
    #assume data is of [ntime nch nstimuli]
    if dim==0:
        data_output=np.zeros([int(data.shape[0]/n),data.shape[1],data.shape[2]])
        for i in range(data_output.shape[0]):
            data_output[i]=np.mean(data[i*n:(i+1)*n],axis=0)
    elif dim ==1:
        data_output=np.zeros([data.shape[0],int(data.shape[1]/n),data.shape[2]])
        for i in range(data_output.shape[1]):
            data_output[:,i,:]=np.mean(data[:,i*n:(i+1)*n,:],axis=1)
    return(data_output)


def featurize_data(input_data, n_train, sampling_type):
    if not 'noparse' in sampling_type:
        if n_train>input_data.shape[2]:
            n_train=input_data.shape[2]
    ### in the [102, 100] case
    if sampling_type == 'spatial_cov':
        inds=np.random.choice(range(input_data.shape[2]),n_train,replace=False)
        features = spatial_cov(input_data[:,:,inds])
        features=np.array(features)
    elif sampling_type == 'spatial_corr':
        inds=np.random.choice(range(input_data.shape[2]),n_train,replace=False)
        features = spatial_corr(input_data[:,:,inds])
        features=np.array(features)
    elif sampling_type == 'temporal_cov':
        inds=np.random.choice(range(input_data.shape[2]),n_train,replace=False)
        features = temporal_cov(input_data[:,:,inds])
        features=np.array(features)    
    elif sampling_type == 'temporal_corr':
        inds=np.random.choice(range(input_data.shape[2]),n_train,replace=False)
        features = temporal_corr(input_data[:,:,inds])
        features=np.array(features)
    elif sampling_type == 'freq_spatial': 
        #[nch  nfreq]
        nt = input_data.shape[1]
        inds=np.random.choice(range(input_data.shape[2]),n_train,replace=False)
        f,t,Sxx =ssg.spectrogram(np.swapaxes(input_data[:,:,inds],1,2), fs=200,nfft=nt,nperseg=nt)
        features= np.squeeze(Sxx.mean(axis=1))
        features=np.array(features)          
    elif sampling_type == 'freq_temporal':
        nt = input_data.shape[1]
        inds=np.random.choice(range(input_data.shape[2]),n_train,replace=False)
        f,t,Sxx =ssg.spectrogram(np.swapaxes(input_data[:,:,inds],1,2), fs=200,noverlap=nt-1,nperseg=nt)
        Sxx=np.swapaxes(Sxx,-1,-2)
        features= np.squeeze(Sxx.mean(axis=1))
        features=np.array(features)
    else:
        print('wrong sampling_type!')
    return(features)

def compare_features(features1, features2, metric='corr',individual = False):
    # compare features1 to features2 (order matters!)
    y_true = np.arange(len(features1))
    if metric=='corr':
        dists = np.corrcoef(features1.reshape(features1.shape[0],-1),features2.reshape(features2.shape[0],-1))[:len(features1),len(features1):]
    else:
        print('unrecognized metric!')
    y_pred=np.argmax(dists,axis=1)
    if individual:
        return(y_pred==y_true)
    else:
        return(np.mean(y_pred==y_true))


def get_acc_results(data, nrep=100,individual = False):
    # return [n_trainsizes, nsession^2, number of nrep]
    accs = []
    for kk in range(int(len(data[0][0])/nrep)): # train size loop
        accs_tmp=[]
        for i in range(len(data)):
            for j in range(len(data)):
                accs_tmp_tmp=[]                
                for k in range(nrep):
                    features1=data[i,:,k+kk*nrep]
                    features2=data[j,:,k+kk*nrep]
                    accs_tmp_tmp.append(compare_features(features1, features2,'corr',individual))
                accs_tmp.append(accs_tmp_tmp) 
        accs.append(accs_tmp)
    return(np.array(accs))


# FST data featurization¶

In [ ]:
def full_featurization(data,savename,train_size, onset = 'after',within=False):
    if onset == 'after':
        data=data[:,:306,201:]
        data=np.swapaxes(data,0,1)
        data=np.swapaxes(data,1,2)
        data = compress_dim_by_n(data,0,3)
        data = ssg.decimate(data,5,axis=1)
    elif onset == 'before':
        data=data[:,:306,:200]
        data=np.swapaxes(data,0,1)
        data=np.swapaxes(data,1,2)
        data = compress_dim_by_n(data,0,3)
        data = ssg.decimate(data,5,axis=1)   
    if within: 
        if data.shape[2]/2<train_size[0]:
            train_size=int(data.shape[2]/2)
            print('n trials too few!')
        datashape = data.shape
        feature1 = []
        feature2 = []
        feature3 = []
        feature1_2 = []
        feature2_2 = []
        feature3_2 = []
        for n_train in train_size:
            for kkk in range(100):
                ind_data1 = np.random.choice(range(data.shape[2]), n_train,replace=False)
                data1 = data[:,:,ind_data1]
                ind_data2 = np.random.choice([x for x in list(range(data.shape[2])) if x not in ind_data1], n_train,replace=False)
                data2 = data[:,:,ind_data2]
                ntimes = data1.shape[1]
                data1=np.reshape(data1,[102,-1],order='F')
                data1=ss.zscore(data1,axis=1)
                data1=np.reshape(data1,[102,ntimes,-1],order='F')
                ntimes = data2.shape[1]
                data2=np.reshape(data2,[102,-1],order='F')
                data2=ss.zscore(data2,axis=1)
                data2=np.reshape(data2,[102,ntimes,-1],order='F')

                sampling_type='spatial_corr'
                features = featurize_data(data1, n_train, sampling_type)
                feature1.append(features)

                sampling_type='temporal_corr'
                features = featurize_data(data1, n_train, sampling_type)
                feature2.append(features)

                sampling_type='freq_spatial'
                features = featurize_data(data1, n_train, sampling_type)
                features=np.mean(features,axis=0)
                feature3.append(features)


                sampling_type='spatial_corr'
                features = featurize_data(data2, n_train, sampling_type)
                feature1_2.append(features)

                sampling_type='temporal_corr'
                features = featurize_data(data2, n_train, sampling_type)
                feature2_2.append(features)


                sampling_type='freq_spatial'
                features = featurize_data(data2, n_train, sampling_type)
                features=np.mean(features,axis=0)
                feature3_2.append(features)
        sio.savemat(savename,mdict={'feature1':feature1,'feature2':feature2,'feature3':feature3,'feature1_2':feature1_2 ,'feature2_2':feature2_2,\
                           'feature3_2':feature3_2,'datashape':datashape})
    else:
    #featurization step
        data=np.reshape(data,[102,-1],order='F')
        data=ss.zscore(data,axis=1)
        data=np.reshape(data,[102,100,-1],order='F')
        feature1 = []
        feature2 = []
        feature3 = []
        for n_train in train_size:
            for kk in range(100):
                #spatial
                sampling_type='spatial_corr'
                features = featurize_data(data, n_train, sampling_type)
                feature1.append(features)
                #temporal
                sampling_type='temporal_corr'
                features = featurize_data(data, n_train, sampling_type)
                feature2.append(features)
                #freq
                sampling_type='freq_spatial'
                features = featurize_data(data, n_train, sampling_type)
                features=np.mean(features,axis=0)
                feature3.append(features)
        datashape = data.shape
        sio.savemat(savename, {'feature1':feature1,'feature2':feature2,'feature3':feature3,'datashape':datashape})


In [ ]:
#creating FST features on datasets of different level of preprocessing
root_dir = "path_to_data";
start = time.time()
subj_names = ['AT','DR','JG','JK']
for subj in subj_names:
    for session in [1,2,3,4]:
        names = glob.glob(root_dir+subj+'/session'+str(session)+'/fst_b*.fif')

        data=np.empty(shape=[0, 306, 701])
        for fname in names:
            data = np.row_stack((data,sio.loadmat(fname+'data.mat')['data']))
        savename=fname[:-6]+'feature_raw.mat'
        train_size=[300]
        full_featurization(data,savename,train_size,'after')
        print(savename)

        data=np.empty(shape=[0, 306, 701])
        for fname in names:
            data = np.row_stack((data,sio.loadmat(fname+'data_sst_empty_filter_ecg_eog.mat')['data']))
        savename=fname[:-6]+'feature_sst_empty_filter_ecg_eog.mat'
        train_size=[1,5,10,25,50,100,200,300]
        full_featurization(data,savename,train_size,'after')     
        print(savename)
        
        savename=fname[:-6]+'feature_sst_empty_filter_ecg_eog.mat'
        train_size=[300]
        full_featurization(data,savename,train_size,'after',True)     
        print(savename)
        

end = time.time()
print(end - start)

# HCP data featurization

In [ ]:
cd 'path_to_data'

In [ ]:
#find common indices between resting and working memory datasets
import re
import glob, os

rmeg = glob.glob("./resting_state/*")
rmeg=[rmeg[i] for  i in range(len(rmeg)) if '.mat' not in rmeg[i]]
rmeg = list([int(re.findall("(\d{6})", rmeg[i])[0]) for i in range(len(rmeg))])

wmeg = glob.glob("./working_memory/*")
wmeg=[wmeg[i] for  i in range(len(wmeg)) if '.mat' not in wmeg[i]]
wmeg = list([int(re.findall("(\d{6})", wmeg[i])[0]) for i in range(len(wmeg))])

print(len(list(set(rmeg) & set(wmeg)) ))
common_ids = np.sort(list(set(rmeg) & set(wmeg)))
common_ch=np.squeeze(sio.loadmat('./ch_inds.mat') ['common_ch'] ) 
common_ch=[common_ch[i].replace(" ", "") for i in range(len(common_ch))]
print(len(common_ch))


In [ ]:
#within-session featurization. Splitting the dataset for each individual into shuffled, non-overlapping trials for training and testing.

def HPC_featurization_within(postfix,train_size,t_begins,t_ends,n_train_rep=25,session_types=[0,1]):
    start = time.time()
    n_common = len(common_ids)
    for session_type in session_types:

        for i in range(len(common_ids)):
            if session_type==0:
                ind_names = glob.glob('./resting_state/'+str(common_ids[i])+'/MEG/Restin/rmegpreproc/*preproc.mat')
            else:
                ind_names = glob.glob('./working_memory/'+str(common_ids[i])+'/MEG/Wrkmem/tmegpreproc/*preproc_TIM.mat')
            ind_sessions = list([int(re.findall("(\d{1})-", ind_names[i])[0]) for i in range(len(ind_names))])
            data_tmp=[]
            for loadname in ind_names:
                mat = sio.loadmat(loadname)
                data=mat['data']
                label_tmp=data[0][0]['label']
                label_tmp=[label_tmp[ii][0][0] for ii in range(len(label_tmp))]
                tmp_mask=np.zeros(len(label_tmp),dtype=bool)
                for jj in range(len(label_tmp)):
                    if label_tmp[jj] in common_ch:
                        tmp_mask[jj]=True
                #featurization
                trials = data[0][0]['trial'][0]
                #data_tmp=ssg.decimate(trials[0][:,10:1010],10,axis=1)

                for iii in range(trials.shape[0]):
                    data_tmp.append(ssg.decimate(trials[iii][tmp_mask,t_begins[session_type]:t_ends[session_type]],5,axis=1))
            data=np.swapaxes(np.swapaxes(np.array(data_tmp),0,1),1,2)
            if data.shape[2]/2<train_size[0]:
                train_size=[int(data.shape[2]/2)]
                print('n trials too few!')
            datashape = data.shape
            feature1 = []
            feature2 = []
            feature3 = []
            feature1_2 = []
            feature2_2 = []
            feature3_2 = []
            for n_train in train_size:
                for kkk in range(n_train_rep):
                    ind_data1 = np.random.choice(range(data.shape[2]), n_train,replace=False)
                    data1 = data[:,:,ind_data1]
                    ind_data2 = np.random.choice([x for x in list(range(data.shape[2])) if x not in ind_data1], n_train,replace=False)
                    data2 = data[:,:,ind_data2]
                    ntimes = data1.shape[1]
                    nch= data1.shape[0]
                    data1=np.reshape(data1,[nch,-1],order='F')
                    data1=ss.zscore(data1,axis=1)
                    data1=np.reshape(data1,[nch,ntimes,-1],order='F')
                    ntimes = data2.shape[1]
                    data2=np.reshape(data2,[nch,-1],order='F')
                    data2=ss.zscore(data2,axis=1)
                    data2=np.reshape(data2,[nch,ntimes,-1],order='F')
                    #spatial
                    sampling_type='spatial_corr'
                    features = featurize_data(data1, n_train, sampling_type)
                    feature1.append(features)
                    #freq spatial

                     #temporal
                    sampling_type='temporal_corr'
                    features = featurize_data(data1, n_train, sampling_type)
                    feature2.append(features)


                    sampling_type='freq_spatial'
                    features = featurize_data(data1, n_train, sampling_type)
                    features=np.mean(features,axis=0)
                    feature3.append(features)


                    sampling_type='spatial_corr'
                    features = featurize_data(data2, n_train, sampling_type)
                    feature1_2.append(features)
                    #freq spatial

                     #temporal
                    sampling_type='temporal_corr'
                    features = featurize_data(data2, n_train, sampling_type)
                    feature2_2.append(features)


                    sampling_type='freq_spatial'
                    features = featurize_data(data2, n_train, sampling_type)
                    features=np.mean(features,axis=0)
                    feature3_2.append(features)
            if session_type==0:
                sio.savemat('./resting_state/'+str(common_ids[i])+'/MEG/Restin/rmegpreproc/'+str(common_ids[i])+'_feature_'+postfix+'.mat',mdict={'feature1':feature1,'feature2':feature2,'feature3':feature3,'feature1_2':feature1_2 ,'feature2_2':feature2_2,'feature3_2':feature3_2,'datashape':datashape})
            else:
                sio.savemat('./working_memory/'+str(common_ids[i])+'/MEG/Wrkmem/tmegpreproc/'+str(common_ids[i])+'_TIM_feature_'+postfix+'.mat',mdict={'feature1':feature1,'feature2':feature2,'feature3':feature3,'feature1_2':feature1_2 ,'feature2_2':feature2_2,'feature3_2':feature3_2,'datashape':datashape})            
            print(str(common_ids[i]))

    end = time.time()
    print(end - start)

In [ ]:
#creating features for the resting-state and working memory (task) datasets for within-session comparison
postfixs=['within_r','within_t' ]
train_size=[200]
t_begins=[0,763] 
t_ends=[1018,763+1018]
n_train_rep=100
for s in range(len(2)):
    session_types=[s]
    postfix = postfixs[s]
    HPC_featurization_within(postfix,train_size,t_begins,t_ends,n_train_rep,session_types)

# MEG-EEG

In [ ]:
EEG = []
subjs = [1,2,3,4,5,6,7,8,10,11,12,13,14,16,18]
for i in subjs:
    data = sio.loadmat('../EEG_no_aspect/Subj'+str(i)+'_EEG_1_110Hz_notch_ica_PPO10POO10_swapped_MEEG_match_ave_alpha15.0_no_aspect.mat')
    EEG.append(data['ave_data'])
    print(data['ave_data'].shape)

MEG = []
subjs = [1,2,3,4,5,6,7,8,10,11,12,13,14,16,18]
for i in subjs:
    data = sio.loadmat('../MEG_no_aspect/Subj'+str(i)+'_1_110Hz_notch_ica_MEEG_match_ave_alpha15.0_no_aspect.mat')
    MEG.append(data['ave_data'])
    print(data['ave_data'].shape)

meg_tp = []
meg_fq = []
n_train_rep=100
ntrain = 200
for i in range(len(MEG)):
    data = MEG[i].swapaxes(0,2).swapaxes(0,1)
    tmp_tp = []
    tmp_fq = []
    for j in range(n_train_rep):
        tmp_tp.append(featurize_data(data, ntrain, 'temporal_corr'))
        tmp_fq.append(np.mean(featurize_data(data, ntrain, 'freq_spatial'),axis=0))
    meg_tp.append(tmp_tp)
    meg_fq.append(tmp_fq)
    print(i)
meg_tp=np.array(meg_tp)
meg_fq=np.array(meg_fq)
meg_tp = np.expand_dims(meg_tp,0)
meg_fq = np.expand_dims(meg_fq,0)


eeg_tp = []
eeg_fq = []

for i in range(len(EEG)):
    data = EEG[i].swapaxes(0,2).swapaxes(0,1)
    tmp_tp = []
    tmp_fq = []
    for j in range(n_train_rep):
        tmp_tp.append(featurize_data(data, ntrain, 'temporal_corr'))
        tmp_fq.append(np.mean(featurize_data(data, ntrain, 'freq_spatial'),axis=0))
    eeg_tp.append(tmp_tp)
    eeg_fq.append(tmp_fq)
    print(i)
eeg_tp=np.array(eeg_tp)
eeg_fq=np.array(eeg_fq)   
eeg_tp = np.expand_dims(eeg_tp,0)
eeg_fq = np.expand_dims(eeg_fq,0)

sio.savemat(savename,mdict={'eeg_tp':eeg_tp,'eeg_fq':eeg_fq,'meg_tp':meg_tp,'meg_fq':meg_fqDD})
